<a href="https://colab.research.google.com/github/Srijita264/HAR-using-image-classification/blob/main/VGG_16_K_Fold_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


This notebook uses 10 fold classification. The VGG-16/2 model has achieved an accuracy of 63%.

In [ ]:
import cv2
import numpy as np
import os
from google.colab.patches import cv2_imshow
import pandas as pd


def read_and_resize_image(image_path, target_size=(500, 500)):
    image = cv2.imread(image_path)

    # Check if the image is successfully loaded
    if image is None:
        print(f"Error loading image: {image_path}")
        return None

    # Check if the image size is valid
    if image.size == 0:
        print(f"Error resizing image: {image_path}")
        return None

    image = cv2.resize(image, target_size)
    return image

# Function to create the mean aggregate image
def create_mean_image(image_paths):
    # Read and resize all images
    images = [read_and_resize_image(image_path) for image_path in image_paths]

    # Convert images to NumPy array
    images_array = np.array(images)

    # Calculate the mean along the first axis (axis=0) to get the aggregate image
    mean_image = np.mean(images_array, axis=0).astype(np.uint8)
    return mean_image

# Function to save the GAF image
def save_gaf_image(gaf_image, label, index):
    folder_path = f"/content/drive/MyDrive/New_Folder_TEST"
    if not os.path.exists(folder_path):
        os.makedirs(folder_path, exist_ok=True)

    # Save the image using OpenCV instead of Matplotlib
    cv2.imwrite(f"{folder_path}/{index}.png", gaf_image)

labels_df = pd.read_csv('/content/drive/MyDrive/y_test.csv', header=None)

for index, row in labels_df.iterrows():
        label = row[0]

        image_paths = [os.path.join('/content/drive/MyDrive/GAF_SUM_TEST_Y_GYRO', f"Class{label}", f"{index+1}.png"), os.path.join('/content/drive/MyDrive/GAF_SUM_TEST_Y_ACC', f"Class{label}", f"{index+1}.png"), os.path.join('/content/drive/MyDrive/GAF_SUM_TEST_Z_GYRO', f"Class {label}", f"{index+1}.png"), os.path.join('/content/drive/MyDrive/GAF_SUM_TEST_Z_ACC', f"Class {label}", f"{index+1}.png"), os.path.join('/content/drive/MyDrive/GAF_SUM_TEST_X_GYRO', f"Class {label}", f"{index+1}.png"), os.path.join('/content/drive/MyDrive/GAF_SUM_TEST_X_ACC', f"Class {label}", f"{index+1}.png")]

        # Create the mean aggregate image
        mean_image = create_mean_image(image_paths)

        # Adjust pixel values for display
        normalized_mean_image = cv2.normalize(mean_image, None, 0, 255, cv2.NORM_MINMAX)
        # Save the mean image using the provided function
        save_gaf_image(normalized_mean_image, label, index+1)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import StratifiedKFold

val_accuracies = []
val_losses = []

# Load the labels without a header
y = pd.read_csv('/content/drive/MyDrive/y_train.csv', header=None)

# Convert labels to string
y = y.astype(str)

# Define the image size and batch size
img_size = (128, 128)  # Adjust the size as per your requirement
batch_size = 32

# Create data generator
datagen = ImageDataGenerator(rescale=1./255)

# Create DataFrame with image paths and labels
df = pd.DataFrame({
    'filename': ['/content/drive/MyDrive/New_Folder/{}.png'.format(i) for i in range(1, len(y) + 1)],
    'label': y[0]
})

# Initialize StratifiedKFold with 10 folds
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Iterate over folds
for fold, (train_index, val_index) in enumerate(skf.split(df['filename'], df['label'])):
    print(f"Training on fold {fold + 1}...")

    # Create train and validation data generators
    train_generator = datagen.flow_from_dataframe(
        df.iloc[train_index],
        x_col='filename',
        y_col='label',
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',  # Assuming it's a classification task
    )

    val_generator = datagen.flow_from_dataframe(
        df.iloc[val_index],
        x_col='filename',
        y_col='label',
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
    )

    # Build the CNN model
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(len(y[0].unique()), activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    history = model.fit(train_generator, epochs=10, validation_data=val_generator)

    # Evaluate the model on the validation set
    val_loss, val_acc = model.evaluate(val_generator)
    print(f'Validation Accuracy on fold {fold + 1}: {val_acc}')

    # Optionally, you can save the model for each fold
    model.save(f'model_fold_{fold + 1}.h5')
    val_loss, val_acc = model.evaluate(val_generator)
    print(f'Validation Accuracy on fold {fold + 1}: {val_acc}')

    # Store performance metrics
    val_accuracies.append(val_acc)
    val_losses.append(val_loss)


mean_val_accuracy = np.mean(val_accuracies)
std_val_accuracy = np.std(val_accuracies)

mean_val_loss = np.mean(val_losses)
std_val_loss = np.std(val_losses)

print(f'Mean Validation Accuracy: {mean_val_accuracy}, Std: {std_val_accuracy}')
print(f'Mean Validation Loss: {mean_val_loss}, Std: {std_val_loss}')
